In [1]:
from glob import glob
from os import path
import xml
import pandas as pd
FOLDER_NAME = "VOCdevkit\\VOC2012\\JPEGImages\\"

In [2]:
import tensorflow

In [199]:
def find_file(folder):
    return glob(path.join(folder,"*.{}".format('xml')))

def text_format(x_min, y_min, x_max, y_max, part_name,file_path, folder_name, img_width, img_height, img_depth):
    x_min = str(int(float(x_min)))
    y_min = str(int(float(y_min)))
    x_max = str(int(float(x_max)))
    y_max = str(int(float(y_max)))
    img_width = str(int(float(img_width)))
    img_height = str(int(float(img_height)))
    img_depth = str(int(float(img_depth)))
    return "{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}\n". format(x_min, y_min, x_max, y_max, part_name,FOLDER_NAME+file_path, folder_name, img_width, img_height, img_depth)

def get_db_desc(annotation_file):
    xml_tree = ET.parse(annotation_file)
    root = xml_tree.getroot()
    folder_name = root.findall('folder')[0].text
    file_path = root.findall('filename')[0].text
    sizes =  root.findall('size')[0]
    img_width = sizes.findall('width')[0].text
    img_height = sizes.findall('height')[0].text
    img_depth = sizes.findall('depth')[0].text
    objects = root.findall('object')
    desc = ""
    for obj in objects:
        part_name = obj.findall('name')[0].text
        bndbox = obj.findall('bndbox')[0]
        x_min = bndbox.findall('xmin')[0].text
        y_min = bndbox.findall('ymin')[0].text
        x_max = bndbox.findall('xmax')[0].text
        y_max = bndbox.findall('ymax')[0].text
        desc +=text_format(x_min, y_min, x_max, y_max, part_name, file_path, folder_name, img_width, img_height, img_depth)
        parts = obj.findall('part')
        for part in parts:
            part_name = part.findall('name')[0].text
            bndbox = part.findall('bndbox')[0]
            x_min = bndbox.findall('xmin')[0].text
            y_min = bndbox.findall('ymin')[0].text
            x_max = bndbox.findall('xmax')[0].text
            y_max = bndbox.findall('ymax')[0].text
            desc +=text_format(x_min, y_min, x_max, y_max, part_name,file_name, folder_name, img_width, img_height, img_depth)
    return desc

In [200]:
def get_full_db_desc():
    annotation_files = find_file('Annotations')
    desc = "xmin,ymin,xmax,ymax,part_name,file_path,folder_name,image_width,image_height,image_depth\n"
    for annotation_file in annotation_files:
        desc +=get_db_desc(annotation_file)
    return desc

In [201]:
desc = get_full_db_desc()

In [202]:
def generate_csv(desc):
    f = open("data.csv", 'w')
    f.write(desc)
    f.close()

In [203]:
generate_csv(desc)

In [204]:
dataset = pd.read_csv("data.csv")

In [205]:
training = dataset.sample(n=int(0.7*len(dataset)))

In [ ]:
training = dataset.sample(n=int(0.7*len(dataset)))

In [206]:
training = training[['file_path','xmin', 'ymin', 'xmax', 'ymax', 'part_name']]

In [207]:
training

,file_path,xmin,ymin,xmax,ymax,part_name
7884,VOCdevkit\VOC2012\JPEGImages\2008_003406.jpg,371,311,392,372,person
38549,VOCdevkit\VOC2012\JPEGImages\2011_004730.jpg,246,61,500,375,person
42986,VOCdevkit\VOC2012\JPEGImages\2012_004019.jpg,310,171,347,241,person
10552,VOCdevkit\VOC2012\JPEGImages\2008_005134.jpg,1,32,469,333,cat
15123,VOCdevkit\VOC2012\JPEGImages\2008_008115.jpg,165,97,344,333,cow
...,...,...,...,...,...,...
35674,VOCdevkit\VOC2012\JPEGImages\2011_002330.jpg,183,98,214,146,person
4887,VOCdevkit\VOC2012\JPEGImages\2007_000027.jpg,241,179,251,191,hand
28718,VOCdevkit\VOC2012\JPEGImages\2010_004385.jpg,25,23,246,226,bird
37115,VOCdevkit\VOC2012\JPEGImages\2011_003044.jpg,105,72,150,119,chair


In [208]:
training.to_csv("training.csv", header=False, index=False)

In [3]:
import cv2

In [17]:
img = cv2.imread('JPEGImages\\2011_003044.jpg')

In [18]:
img.shape

(375, 500, 3)